In [1]:
import pickle as pkl
import itertools as it
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import gurobipy as gbp
%matplotlib inline

In [134]:
class RouterAllocation(object):
    
    def __init__(self, instance, update_s=False):
        """
        Inicializa a classe, recebendo uma configuração de instancia.
        
        input: dict 
            like: 
            {
                'router_types': router,
                'router_spots': grid_router,
                'client_spots': grid_clients,
                'client_min_sig': Ak,
                'name': 'instance0'
            }
        """
        self.update = update_s
        
        self.Ak           = instance['client_min_sig']
        self.grid_router  = instance['router_spots']
        self.grid_clients = instance['client_spots']
        self.routers      = instance['router_types']
        self.name         = instance['name']
        
        self.r_j = len(self.grid_router)
        self.r_k = len(self.grid_clients)
        self.r_i = len(self.routers.keys())
        
        self.compute_contributions()
        self.generate_model()
        print('Pronto pra rodar!')
    
    def signal_strength(self, xe, ye, xr, yr, router):
        d = np.linalg.norm([[xe - xr], [ye - yr]])
        return router['k'] / np.square(d)

    def run(self):
        self.m.optimize()
        for v in self.m.getVars():
            if v.x: print(v.varName, v.x)
        print('Obj:', self.m.objVal)
        self.plot_solution()
    
    def compute_contributions(self):
        print('Calculando contribuição dos roteadores...')
        try:
            self.Sijk = pkl.load(open("Sijk_%s.p" % self.name, "rb"))
        except:
            self.Sijk = np.array([[[self.signal_strength(xe, ye, xr, yr, router) if (xe != xr) & (ye != yr) else 1 for xr, yr in self.grid_clients] for xe, ye in self.grid_router] 
                                  for n, router in self.routers.items()])
            pkl.dump(self.Sijk, open("Sijk_%s.p" % self.name, "wb"))
        finally:
            if self.update:
                self.Sijk = np.array([[[self.signal_strength(xe, ye, xr, yr, router) if (xe != xr) & (ye != yr) else 1 for xr, yr in self.grid_clients] for xe, ye in self.grid_router] 
                                  for n, router in self.routers.items()])
                pkl.dump(self.Sijk, open("Sijk_%s.p" % self.name, "wb"))
                
        
    def plot_solution(self):
        
        Rij = np.array([v.x for v in self.m.getVars()]).reshape((r_i, r_j))
        
        client_points_x = len(np.unique([a[0] for a in self.grid_clients]))
        client_points_y = len(np.unique([a[1] for a in self.grid_clients]))
        
        potencia = np.array([np.log10(np.sum([Rij[i, j] * self.Sijk[i, j, k] for j in range(self.r_j) for i in range(self.r_i)])) for k in range(self.r_k)]).reshape(client_points_x, client_points_y)

        fig, ax = plt.subplots(figsize=(10,10))
        c = ax.pcolor(potencia)
        fig.colorbar(c, ax=ax)
        ax.set_title('Instancia %s' % self.name)
        ax.set_xlabel('Coordenada X (m)')
        ax.set_ylabel('Coordenada Y (m)');
    
    def generate_model(self):
        """
        Monta o modelo com os dados da instancia.

        """
        print('Gerando modelo...')
        self.m = gbp.Model("Alocação de roteadores")

        # variaveis
        Xij = np.array([[self.m.addVar(vtype=gbp.GRB.BINARY, name="X %d,%d" % (i, j)) for j in range(self.r_j)] for i in range(self.r_i)])

        # função objetivo
        Ci = [r['c'] for r in self.routers.values()]

        obj = gbp.LinExpr()
        for j in range(self.r_j): 
            for i in range(self.r_i): 
                obj += Ci[i] * Xij[i, j]
        self.m.setObjective(obj, gbp.GRB.MINIMIZE)

        ## restrições
        # maximo 1 roteador por ponto no grid
        for j in range(self.r_j):
            rest = gbp.LinExpr()
            for i in range(self.r_i): rest += Xij[i, j]
            self.m.addConstr(rest <= 1, "r %d" % j)


        # restrição de sinal
        for k in range(self.r_k): 
            rest = gbp.LinExpr()
            for i in range(self.r_i): 
                for j in range(self.r_j):
                    rest += Xij[i, j] * self.Sijk[i, j, k]
            self.m.addConstr(rest >= self.Ak[k], "s %d" % k)

### Gerar instancias

In [143]:
# area
x = 200
y = 200

# 
router_points_x = 20
router_points_y = 20

client_points_x = 20
client_points_y = 20


r_j = router_points_x * router_points_y
r_k = client_points_x * client_points_y
r_i = len(routers.keys())

grid_router = list(it.product(np.linspace(0, x-1, router_points_x, dtype=np.int), np.linspace(0, y-1, router_points_y, dtype=np.int)))
grid_clients = list(it.product(np.linspace(0, x-1, client_points_x, dtype=np.int)+1, np.linspace(0, y-1, client_points_y, dtype=np.int)+1))

# minimos
dB = -60
Ak = [10 ** (dB/20) for i in range(len(grid_clients))]

# tipos de roteador
routers = {
    0: {'k': 10, 'c': 100},
    1: {'k': 20, 'c': 200},
    2: {'k': 30, 'c': 300},
    3: {'k': 40, 'c': 400},
    4: {'k': 50, 'c': 500},
    5: {'k': 60, 'c': 600}
}

instancia0 = {
    'router_types': routers,
    'router_spots': grid_router,
    'client_spots': grid_clients,
    'client_min_sig': Ak,
    'name': 'instancia0'
}

router_points_x = 10
router_points_y = 10

client_points_x = 20
client_points_y = 20


r_j = router_points_x * router_points_y
r_k = client_points_x * client_points_y
r_i = len(routers.keys())

grid_router = list(it.product(np.linspace(0, x-1, router_points_x, dtype=np.int), np.linspace(0, y-1, router_points_y, dtype=np.int)))
grid_clients = list(it.product(np.linspace(0, x-1, client_points_x, dtype=np.int)+1, np.linspace(0, y-1, client_points_y, dtype=np.int)+1))

# minimos
dB = -40
Ak = [10 ** (dB/20) for i in range(len(grid_clients))]
instancia = {
    'router_types': routers,
    'router_spots': grid_router,
    'client_spots': grid_clients,
    'client_min_sig': Ak,
    'name': 'instancia1'
}


# minimos
dB = -60
Ak = [10 ** (dB/20) for i in range(len(grid_clients))]
routers = {
    0: {'k': 10, 'c': 100},
    1: {'k': 20, 'c': 120},
    2: {'k': 30, 'c': 140},
    3: {'k': 40, 'c': 160},
    4: {'k': 50, 'c': 170},
    5: {'k': 60, 'c': 200}
}

instancia2 = {
    'router_types': routers,
    'router_spots': grid_router,
    'client_spots': grid_clients,
    'client_min_sig': Ak,
    'name': 'instancia2'
}



# 
router_points_x = 10
router_points_y = 10

client_points_x = 20
client_points_y = 20


r_j = router_points_x * router_points_y
r_k = client_points_x * client_points_y
r_i = len(routers.keys())

grid_router = list(it.product(np.linspace(0, x-1, router_points_x, dtype=np.int), np.linspace(0, y-1, router_points_y, dtype=np.int)))
grid_clients = list(it.product(np.linspace(0, x-1, client_points_x, dtype=np.int)+1, np.linspace(0, y-1, client_points_y, dtype=np.int)+1))
dB = -60
Ak = [10 ** (dB/20) for i in range(len(grid_clients))]
instancia3 = {
    'router_types': routers,
    'router_spots': grid_router,
    'client_spots': grid_clients,
    'client_min_sig': Ak,
    'name': 'instancia3'
}


Calculando contribuição dos roteadores...
Gerando modelo...
Pronto pra rodar!
Optimize a model with 800 rows, 2400 columns and 962400 nonzeros
Variable types: 0 continuous, 2400 integer (2400 binary)
Coefficient statistics:
  Matrix range     [1e-04, 3e+01]
  Objective range  [1e+02, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-03, 1e+00]
Found heuristic solution: objective 500.0000000
Presolve removed 0 rows and 554 columns
Presolve time: 4.30s
Presolved: 800 rows, 1846 columns, 740246 nonzeros
Variable types: 0 continuous, 1846 integer (1846 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.048000e+02   0.000000e+00      5s
      10    1.6769296e+02   0.000000e+00   0.000000e+00      5s

Root relaxation: objective 1.676930e+02, 10 iterations, 0.20 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | I

In [137]:
opt = RouterAllocation(instancia0)
opt.run()

Calculando contribuição dos roteadores...
Gerando modelo...
Pronto pra rodar!
Optimize a model with 500 rows, 600 columns and 240600 nonzeros
Variable types: 0 continuous, 600 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e-04, 3e+01]
  Objective range  [1e+02, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-02, 1e+00]
Found heuristic solution: objective 2100.0000000
Presolve time: 0.49s
Presolved: 500 rows, 600 columns, 240600 nonzeros
Variable types: 0 continuous, 600 integer (600 binary)

Root relaxation: objective 6.070237e+02, 61 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  607.02373    0   20 2100.00000  607.02373  71.1%     -    0s
H    0     0                     800.0000000  607.02373  24.1%     -    0s
     0     0  618.86514    0   25  800.00000  618.86514  22.6%     -    1s
     0     0  618.865

In [144]:
opt = RouterAllocation(instancia, update_s=True)
opt.run()

Calculando contribuição dos roteadores...
Gerando modelo...
Pronto pra rodar!
Optimize a model with 800 rows, 2400 columns and 962400 nonzeros
Variable types: 0 continuous, 2400 integer (2400 binary)
Coefficient statistics:
  Matrix range     [1e-04, 3e+01]
  Objective range  [1e+02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-03, 1e+00]
Found heuristic solution: objective 170.0000000
Presolve removed 0 rows and 554 columns
Presolve time: 3.57s
Presolved: 800 rows, 1846 columns, 740246 nonzeros
Variable types: 0 continuous, 1846 integer (1846 binary)

Root relaxation: objective 1.227326e+02, 7 iterations, 0.11 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  122.73263    0    4  170.00000  122.73263  27.8%     -    4s
H    0     0                     140.0000000  122.73263  12.3%     -    4s
     0     0     cutoff    0       140.00000  140.00

In [140]:
opt = RouterAllocation(instancia2)
opt.run()

Calculando contribuição dos roteadores...
Gerando modelo...
Pronto pra rodar!
Optimize a model with 500 rows, 600 columns and 240600 nonzeros
Variable types: 0 continuous, 600 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e-04, 3e+01]
  Objective range  [1e+02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-03, 1e+00]
Found heuristic solution: objective 290.0000000
Presolve removed 0 rows and 86 columns
Presolve time: 0.64s
Presolved: 500 rows, 514 columns, 206114 nonzeros
Variable types: 0 continuous, 514 integer (514 binary)

Root relaxation: objective 1.261801e+02, 8 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  126.18006    0    4  290.00000  126.18006  56.5%     -    0s
H    0     0                     220.0000000  126.18006  42.6%     -    0s
H    0     0                     140.0000000  126.18006  9.

In [142]:
opt = RouterAllocation(instancia3)
opt.run()